In [5]:
from myclasses.sql_executor import SQLExecutor
import pandas as pd

if __name__ == "__main__":
    # Create an instance of SQLExecutor
    sql_executor = SQLExecutor()

    # Example SQL query
    query = """
        SELECT 
            (net_income / num_beds) as net_income,
            year,
            tot_fund_balance,
            ownership,
            acct_payable,
            acct_rec,
            total_liabilities,
            current_ratio,
            quick_ratio,
            (tot_days / tot_bed_days_avail) as fill_rate,
            overhead_nonsalary_costs,
            tot_salaries,
            cash,
            chow_last_12mos,
            region,
            state,
            state_lean,
            county_ssa,
            zip,
            tot_discharge_tot,
            def_score,
            fine_tot,
            fine_cnt,
            resfamcouncil,
            sprinkler_status,
            overall_rating,
            quality_rating,
            staffing_rating,
            rn_staffing_rating,
            aidhrd,
            vochrd,
            rnhrd,
            totlichrd,
            tothrd,
            pthrd,
            weighted_all_cycles_score,
            certification,
            snf_avg_stay_len_title_tot,
            pop_over_70,
            over_70_pct,
            has_outpatient,
            bedcert,
            contract_labor
            
        FROM Master
        WHERE year in (2020, 2021, 2022)
        ORDER BY prov_id;
        """

    # Execute the query and get the result as a DataFrame
    df = sql_executor.execute_query(query)

# Assuming NA contract_labor means the Nursing home does have contract_labor
df[['contract_labor']] = df[['contract_labor']].fillna(0)

In [6]:
df.describe()

,net_income,year,tot_fund_balance,acct_payable,acct_rec,total_liabilities,current_ratio,quick_ratio,fill_rate,overhead_nonsalary_costs,...,rnhrd,totlichrd,tothrd,pthrd,weighted_all_cycles_score,snf_avg_stay_len_title_tot,pop_over_70,over_70_pct,bedcert,contract_labor
count,3.666900e+04,36669.000000,3.666900e+04,3.666900e+04,3.666900e+04,3.666900e+04,36669.000000,36669.000000,36669.000000,3.666900e+04,...,36669.000000,36669.000000,36669.000000,36669.000000,28048.000000,36669.000000,36669.000000,36669.000000,36669.000000,36669.000000
mean,1.244034e+03,2020.814421,3.099135e+04,8.145473e+03,1.817111e+04,1.277870e+05,10.474222,5.640781,0.725666,5.628024e+04,...,0.680087,1.581166,3.875040,0.073617,64.156051,2.201931,67758.346014,0.119606,0.986599,7237.352978
std,2.655835e+04,0.758252,3.091946e+05,4.522035e+04,1.270708e+05,6.512118e+05,1093.324057,652.646874,2.580607,5.619001e+04,...,0.363639,0.430981,0.785252,0.069285,67.555344,5.896135,150044.498309,0.031257,0.299421,8853.565609
min,-1.700283e+06,2020.000000,-9.221566e+06,-1.550743e+06,-4.259891e+05,-1.621231e+06,-816.141414,-4793.230126,0.002740,5.767761e+01,...,0.000000,0.000000,1.503410,0.000000,0.000000,0.006042,84.000000,0.030065,0.000840,0.187726
25%,-4.442894e+03,2020.000000,-3.767515e+03,1.744122e+03,6.210200e+03,9.168044e+03,0.667167,0.006828,0.606571,3.531894e+04,...,0.489300,1.388810,3.466260,0.038890,22.000000,0.727882,3303.000000,0.101151,1.000000,3314.414141
50%,1.636360e+03,2021.000000,8.707318e+03,3.748097e+03,9.974263e+03,2.066852e+04,1.277315,0.137777,0.724408,4.745570e+04,...,0.642225,1.508395,3.810080,0.067630,44.000000,1.243656,16434.000000,0.118884,1.000000,5510.663043
75%,8.427323e+03,2021.000000,3.161471e+04,7.994560e+03,1.543454e+04,5.677432e+04,2.492608,0.812930,0.821013,6.211382e+04,...,0.754160,1.677150,4.067870,0.080900,82.000000,2.229352,66948.000000,0.135104,1.000000,9087.793814
max,1.401623e+06,2022.000000,1.869957e+07,2.027600e+06,1.701048e+07,2.896913e+07,203696.688525,119671.689243,321.645833,2.146148e+06,...,7.384050,7.929440,16.326920,1.450440,1118.000000,175.703125,973386.000000,0.453558,5.555556,825561.760417


In [7]:
def bin_income(data):

    if data >= 6000:
        return '5'
    elif data >= 3000:
        return '4'
    elif data >= 1000:
        return '3'
    elif data >= -1000:
        return '2'
    elif data >= -4000:
        return '1'
    else:
        return '0'
    
df['net_income'] = df['net_income'].apply(bin_income)

In [8]:
from sklearn.model_selection import train_test_split

y = df['net_income']
df = df.drop(columns='net_income')

# Select columns where data type is text (object)
text_columns = df.select_dtypes(include=['object']).columns

# Create dummy variables for text columns
df_encoded = pd.get_dummies(df, columns=text_columns)

# Convert all columns to numeric type
df_encoded = df_encoded.apply(pd.to_numeric, errors='coerce')

# Extract target variable (y) and independent variables (X)
X = df_encoded.iloc[:, 2:].fillna(0)

In [9]:
# import umap.umap_ as umap

# # Assuming X_train is your training data

# # Define UMAP reducer
# reducer = umap.UMAP(n_components=8)

# # Fit and transform the training data
# umap_embeddings = reducer.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

num_classes = 6

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode target values into integers
y_train_encoded = label_encoder.fit_transform(y_train)

# Convert integer target values to one-hot encoded format
num_classes = len(label_encoder.classes_)
y_train_encoded = to_categorical(y_train_encoded, num_classes=num_classes)

# Define the architecture of the neural network
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Input layer
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(2, activation='relu'),
    Dense(num_classes, activation='softmax')  # Change activation function for multi-classification
])

# Compile the model with categorical_crossentropy loss
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Change loss and metrics

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train_encoded, epochs=30, batch_size=6, validation_split=0.2, callbacks=[early_stop])

2024-04-24 11:27:07.496685: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-24 11:27:07.497229: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-24 11:27:07.502557: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-24 11:27:07.561725: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 11:27:09.191969: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

Epoch 1/30
2934/2934 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.3105 - loss: 1.9398 - val_accuracy: 0.3188 - val_loss: 1.6557
Epoch 2/30
2934/2934 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - accuracy: 0.3179 - loss: 1.6597 - val_accuracy: 0.3188 - val_loss: 1.6556
Epoch 3/30
2934/2934 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - accuracy: 0.3211 - loss: 1.6566 - val_accuracy: 0.3188 - val_loss: 1.6559
Epoch 4/30
2934/2934 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.3186 - loss: 1.6607 - val_accuracy: 0.3188 - val_loss: 1.6556
Epoch 5/30
2934/2934 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.3180 - loss: 1.6609 - val_accuracy: 0.3188 - val_loss: 1.6556
Epoch 6/30
2934/2934 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.3249 - loss: 1.6563 - val_accuracy: 0.3188 - val_loss: 1.6555
Epoch 7/30
2934/2934 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.3216 - loss: 1.6622 - val_accuracy: 0.3188 - val_loss: 1.6556
Epoch 8/30
2934/2934 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.3309 - loss: 1

In [ ]:
import numpy as np
from sklearn.metrics import r2_score, median_absolute_error, max_error, mean_absolute_error, mean_squared_error, explained_variance_score

# Apply the model on test data
y_pred = model.predict(X_test)

# R² Score (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)

# Median Absolute Error
mabe = median_absolute_error(y_test, y_pred)

# Max Error
max_error_value = max_error(y_test, y_pred)

# Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)

# Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

# Root Mean Squared Error
rmse = np.sqrt(mse)

# Explained Variance Score
explained_variance = explained_variance_score(y_test, y_pred)


print("Root Mean Squared Error: ${:.2f}".format(rmse))
print("Median Absolute Error: ${:.2f}".format(mabe))
print("Mean Absolute Error: ${:.2f}".format(mae))
print("Max Error: ${:.2f}".format(max_error_value))
print("R² Score (Coefficient of Determination): {:.5f}".format(r2))

In [ ]:
import matplotlib.pyplot as plt

# Convert values to thousands
y_test_thousands = y_test / 1000
y_pred_thousands = y_pred / 1000

plt.figure(figsize=(8, 6))
plt.scatter(y_test_thousands, y_pred_thousands, color='blue', alpha=0.5, s=1)  # Scatter plot of actual vs. predicted values with smaller point size
plt.plot([y_test_thousands.min(), y_test_thousands.max()], [y_test_thousands.min(), y_test_thousands.max()], 'k--', lw=1)  # Plot the diagonal line
plt.xlabel('Actual (thousands)')
plt.ylabel('Predicted (thousands)')
plt.title('Actual vs. Predicted Values')

# Set axis limits to show the actual values
plt.xlim(left=y_test_thousands.min(), right=y_test_thousands.max())
plt.ylim(bottom=y_test_thousands.min(), top=y_test_thousands.max())

# Add light gridlines
plt.grid(color='gray', linestyle='--', linewidth=0.5)

plt.show()